## Stats507 Homework 3, Winter 2020
### Moeki Kurita
#### mkurita@umich.edu

__Remark: For this homework, do not use "input" in your code!!!__

Problem 1 took me 1.5 hour, Problem 2 - 3 hours, Problem 3 - 1.5 hour, Problem 4 - 45 minutes, Problem 5 - 1.5 hour, Problem 6 - 30 minutes

In [1]:
# you might need to install pycodestyle_magic and flake8
%load_ext pycodestyle_magic
%pycodestyle_on

### Problem 1: Counting Word Bigrams
1. Write a function count_bigrams_in_file that takes a filename as its only argument.

In [2]:
import string


def count_bigrams_in_file(filename):
    try:
        if type(filename) != str:
            raise TypeError
        with open(filename, 'r') as file_obj:
            lines = file_obj.readlines()
            words = []
            for line in lines:
                words.extend(line.translate(str.maketrans(
                    '', '', string.punctuation)).strip().lower().split())
        count_dict = dict()
        for i in range(len(words)-1):
            bigram = (words[i], words[i+1])
            if bigram in count_dict:
                count_dict[bigram] += 1
            else:
                count_dict[bigram] = 1
        return count_dict
    except TypeError:
        raise TypeError('The argument must be a string')
    except FileNotFoundError:
        raise FileNotFoundError("The given file cannot be found")

In [3]:
with open('./test.txt', 'w') as f:
    f.write('Half a league, half a league,\n Half a league onward.')

count_bigrams_in_file('test.txt') == {('a', 'league'): 3,
                                      ('half', 'a'): 3,
                                      ('league', 'half'): 2,
                                      ('league', 'onward'): 1}

True

2. Run your function on this file, and pickle the resulting dictionary in a file called mb.bigrams.pickle.

In [4]:
import pickle

bigram_dict = count_bigrams_in_file("WandP.txt")

with open("mb.bigrams.pickle", "wb") as file_obj:
    pickle.dump(bigram_dict, file_obj)

3. Write a function collocations that takes a file name as its only argument and returns a dictionary.

In [5]:
def collocations(filename):
    try:
        if type(filename) != str:
            raise TypeError
        with open(filename, 'r') as file_obj:
            lines = file_obj.readlines()
            words = []
            for line in lines:
                words.extend(line.translate(str.maketrans(
                    '', '', string.punctuation)).strip().lower().split())
        all_bigrams_list = []
        for i in range(len(words)-1):
            bigram = (words[i], words[i+1])
            all_bigrams_list.append(bigram)
        result_dict = dict()
        for tpl in all_bigrams_list:
            for i in tpl:
                result_dict[i] = result_dict.get(i, set())
            result_dict[tpl[0]].add(tpl[1])
            result_dict[tpl[1]].add(tpl[0])
        return result_dict
    except TypeError:
        raise TypeError('The argument must be a string')
    except FileNotFoundError:
        raise FileNotFoundError("The given file cannot be found")

In [6]:
collocations('test.txt') == {'half': {'league', 'a'},
                             'a': {'half', 'league'},
                             'league': {'onward', 'half', 'a'},
                             'onward': {'league'}}

True

4. Run your function on the file WandP.txt and pickle the resulting dictionary in a file called mb.colloc.pickle.

In [7]:
collocations_dict = collocations("WandP.txt")

with open("mb.colloc.pickle", "wb") as file_obj:
    pickle.dump(collocations_dict, file_obj)

### Problem 2: More Fun with Vectors
#### Define the class Vector in one code cell below, since class can't be declared several times.

##### Answer for (2):
I chose tuples, because tuples are immutable. When we make calculations between vectors (sum, product, etc.), original values should be protected from being modified. If not protected, the results of the same calculations may change everytime, which is not desirable.
##### Answer for (3):
I chose instance attributes. Class attributes will be shared across all instances of the class. We want to use various vectors with different dimensions or entries in a program, so each vector instance should have its own value.

In [8]:
class Vector:
    def __init__(self, dimension, entries=None):
        if not isinstance(dimension, int):
            raise TypeError('Dimension must be an integer')
        if dimension < 0:
            raise ValueError('Dimension cannot be a negative number')
        if entries is None:
            entries = [0]*dimension
        if dimension != len(entries):
            raise ValueError('Dimension and number of entries must be same')
        if not all(isinstance(i, (int, float)) for i in entries):
            raise TypeError('All entries must be an integer or float')
        self.d = dimension
        self.e = tuple(entries)

    def __eq__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        if self.d != other.d:
            raise ValueError("Same dimension required")
        return self.e == other.e

    def __lt__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        if self.d != other.d:
            raise ValueError("Same dimension required")
        return self.e < other.e

    def __ne__(self, other):
        return not self == other

    def __le__(self, other):
        return self < other or self == other

    def __gt__(self, other):
        return not self < other

    def __ge__(self, other):
        return not self < other

    def dot(self, other):
        if not isinstance(other, Vector):
            raise TypeError("The argument must be a vector")
        if self.d != other.d:
            raise ValueError("Same dimension required")
        summed = sum(self.e[i]*other.e[i] for i in range(self.d))
        return summed

    def __mul__(self, other):
        if isinstance(other, Vector):
            if self.d == other.d:
                return Vector(self.d, [x*y for x, y in zip(self.e, other.e)])
            else:
                raise ValueError("Same dimension required")
        elif isinstance(other, (int, float)):
            return Vector(self.d, [x*2 for x in self.e])
        else:
            raise TypeError("Multiplier must be a vector or int/float")

    def __rmul__(self, other):
        return self*other

    def norm(self, p):
        if not isinstance(p, (int, float)):
            raise TypeError("The input must be an int/float")
        elif p < 0:
            raise ValueError("Negative-norm has not been defined")
        elif p == 0:
            return sum(1 for i in self.e if i != 0)
        elif p != float("inf"):
            return pow(sum(abs(i)**p for i in self.e), 1/p)
        elif p == float("inf"):
            return max(abs(i) for i in self.e)

In [9]:
# 2.1
print(isinstance(Vector(3, (1, 2, 3)), Vector))

# 2.4
print((Vector(3, (1, 2, 3)) > Vector(3, (1, 2, 2))) == (True))
print((Vector(3, (1, 2, 3)) >= Vector(3, (1, 2, 2))) == (True))
print((Vector(3, (1, 2, 3)) < Vector(3, (1, 2, 2))) == (False))
print((Vector(3, (1, 2, 3)) <= Vector(3, (1, 2, 2))) == (False))
print((Vector(3, (1, 2, 3)) == Vector(3, (1, 2, 2))) == (False))

# 2.5
print((Vector(3, (1, 2, 3)).dot(Vector(3, (1, 2, 2)))) == 11.0)

# 2.6
print(2 * Vector(3, (1, 2, 3)) == Vector(3, (2, 4, 6)))
print(Vector(3, (1, 2, 3)) * 2 == Vector(3, (2, 4, 6)))

# 2.7
print(Vector(3, (1, 2, 3)).norm(1) == 6.0)

True
True
True
True
True
True
True
True
True
True


###  Problem 3: Iterators and Generators
1. Define a class Fibo of iterators that enumerate the Fibonacci numbers.

In [10]:
class Fibo():
    def __init__(self):
        self.a, self.b = 0, 1

    def __iter__(self):
        return self

    def __next__(self):
        yielded = self.a
        self.a, self.b = self.b, self.a + self.b
        return yielded

In [11]:
ans = Fibo()
[next(ans) for _ in range(10)] == [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

True

2. Define a class GenFibo of iterators that enumerate generalized Fibonacci numbers.

In [12]:
class GenFibo(Fibo):
    def __init__(self, f_0=0, f_1=1):
        self.a, self.b = f_0, f_1

In [13]:
ans = GenFibo(2, 1)
[next(ans) for _ in range(10)] == [2, 1, 3, 4, 7, 11, 18, 29, 47, 76]

True

3. Define a generator primes that enumerates the prime numbers.
#### You may use the function is_prime that we defined in class (or something similar to it), but such solutions will not receive full credit

In [14]:
def primes(p=2):
    while True:
        while any(p % x == 0 for x in range(2, p)):
            p += 1
        yield p
        p += 1

In [15]:
ans = primes()
[next(ans) for _ in range(10)] == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

True

4. Define a generator ulam that enumerates the Ulam numbers.

In [16]:
import itertools as itls


def whether_distinct(l, i):
    ways = 0
    ways += sum(1 for c in itls.combinations(l, 2) if i == sum(c))
    return ways == 1


def ulam(u=1):
    t = []
    while True:
        if u <= 2:
            yield u
            t.append(u)
            u += 1
        elif u > 2:
            if whether_distinct(t, u):
                yield u
                t.append(u)
                u += 1
            else:
                u += 1

In [17]:
ans = ulam()
[next(ans) for _ in range(10)] == [1, 2, 3, 4, 6, 8, 11, 13, 16, 18]

True

###  Problem 4: List Comprehensions and Generator Expressions (5 points)
1. Write a list comprehension that enumerates the sequence $3^n - 1$ for n = 1, 2, 3, ..., 20.

In [18]:
pow3minus1 = [(3**n)-1 for n in range(1, 21)]

2. Write a generator expression that enumerates the Lazy Caterer's sequence. Please assign this generator expression to a variable called caterer.

In [19]:
def non_neg_gen():
    i = 0
    while True:
        yield i
        i += 1


caterer = (((n**2)+n+2)/2 for n in non_neg_gen())

3. Write a generator expression that enumerates the pyramid numbers.

In [20]:
def pos_gen():
    i = 1
    while True:
        yield i
        i += 1


pyramid = ((n*(n+1)*(2*n+1))/6 for n in pos_gen())

4. Write a generator expression that enumerates the octahedral numbers.

In [21]:
def pyra_gen(n):
    return (n*(n+1)*(2*n+1))/6


octa = (sum(map(pyra_gen, [n, n-1])) for n in pos_gen())

### Problem 5: Map, Filter and Reduce
1. Write a one-line expression that computes the sum of the first 10 odd  (starting from 1).

In [1]:
import functools as ftls

sum_of_odd_numbers = ftls.reduce(lambda x, y: x+y,
                                 filter(lambda n: n % 2 != 0, range(20)), 0)

2. Write a one-line expression that computes the product of the first 13 primes. You may use the primes generator that you defined above. Please assign the output of this expression to a variable called product_of_primes.

In [23]:
prm_gen = primes()

product_of_primes = ftls.reduce(lambda x, y: x*y,
                                [next(prm_gen) for _ in range(13)], 1)
print(product_of_primes)

304250263527210


3. Write a one-line expression that computes the sum of the squares of the first 31 primes. You may use the primes generator that you defined above. Please assign the output of this expression to a variable called squared_primes.

In [1]:
prm_gen2 = primes()

squared_primes = ftls.reduce(lambda x, y: x+y,
                             map(lambda x: x**2,
                                 [next(prm_gen2) for _ in range(31)]), 0)

NameError: name 'primes' is not defined

4. Write a one-line expression that computes a list of the first twenty harmonic numbers. Please assign the output of this expression to a variable called harmonics.

In [25]:
harmonics = [i for i in itls.accumulate([1/k for k in range(1, 21)])]

5. Write a one-line expression that computes the geometric mean of the first 12 octahedral numbers.

In [26]:
octa2 = (sum(map(pyra_gen, [n, n-1])) for n in pos_gen())

octa_geom = (ftls.reduce(lambda x, y: x*y,
                         [next(octa2) for _ in range(12)], 1))**(1/12)
print(octa_geom)

104.36220448693885


### Problem 4: Fun with Polynomials
1. Write a function make_poly that takes a list of numbers (ints and/or foats) coeffs as its only argument and returns a function p.

In [27]:
def make_poly(coeffs):
    def p(x):
        return ftls.reduce(lambda a, b: a+b,
                           [coeffs[i]*(x**i) for i in range(len(coeffs))], 0)
    return p

In [28]:
make_poly([1, 3, 4])(2) == 23

True

2. Write a function eval_poly that takes two lists of numbers (ints and/or foats), coeffs and args.

In [29]:
def eval_poly(coeffs, args):
    return [make_poly(coeffs)(elm) for elm in args]

In [30]:
eval_poly([1, 3, 4], [2, 3]) == [23, 46]

True